<a href="https://colab.research.google.com/github/pichayaanik/Capstone_PichayaAnik/blob/main/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Discovering Key-Drivers for Boosting Thailand's Digital Competitiveness Index Using an NLP Approach**
---
Pichaya Charoonpongsakdi (Anik) <br>
GA-TDA DSI Capstone Project

# Part 1: Problem Statement
---

#Part 2: Data Wrangling
---


##2.0 Install and Import Libraries

In [ ]:
# Install pythainal library
!pip install pythainlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.6 MB 8.0 MB/s 


In [ ]:
# Import Basic Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import Sklearn Libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Import NLP Libraries
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
from pythainlp.tokenize import sent_tokenize, word_tokenize
from pythainlp.corpus import thai_stopwords
import re

##2.1 Load Datasets

In [ ]:
# Download datasets from Google Drive
!gdown 1BQtzBne72ML1OoJG46IbYQ_v9pUrOq1K

Downloading...
From: https://drive.google.com/uc?id=1BQtzBne72ML1OoJG46IbYQ_v9pUrOq1K
To: /content/bkkbiz_update.json
100% 2.29G/2.29G [00:16<00:00, 141MB/s]


In [ ]:
# Import datasets
bkkbiz = pd.read_json('/content/bkkbiz_update.json')

In [ ]:
# Check head
bkkbiz.head()

,ID,date,category,title,description,article,url
0,620001,2014-11-27 08:01:00,ต่างประเทศ,น้ำมันดิบปิดร่วง-คาดโอเปคไม่ลดกำลังผลิต,น้ำมันดิบปิดร่วงแตะระดับต่ำสุดในรอบ 4 ปี-คาดโอ...,สัญญาน้ำมันดิบตลาดล่วงหน้าสหรัฐ ปิดตลาดปรับตัว...,https://www.bangkokbiznews.com/news/detail/620001
1,620002,2014-11-27 08:20:00,ต่างประเทศ,ทองคำปิดร่วงหลังสหรัฐเผยข้อมูลเศรษฐกิจ,สัญญาทองคำตลาดล่วงหน้าสหรัฐ ปิดปรับตัวลงเล็กน้...,สัญญาทองคำตลาดล่วงหน้าสหรัฐ ปิดปรับตัวลงเล็กน้...,https://www.bangkokbiznews.com/news/detail/620002
2,620009,2014-11-27 09:34:00,การเงิน-การลงทุน,"ทองแท่งขายออก18,600บ.ไม่เปลี่ยนแปลง","ราคาทองคำวันนี้ทองแท่งขายออกบาทละ 18,600 บาท ท...",สมาคมค้าทองคำรายงานราคาทอง 96.5% ประจำวันที่ 2...,https://www.bangkokbiznews.com/news/detail/620009
3,620010,2014-11-27 09:36:00,การเงิน-การลงทุน,เงินบาทเปิด32.75/77คาดแกว่งแคบ,ค่าเงินบาทเปิดที่ 32.75/77 บาท/ดอลลาร์ คาดแกว่...,นักบริหารเงินจากธนาคารพาณิชย์ เปิดเผยว่า เงินบ...,https://www.bangkokbiznews.com/news/detail/620010
4,620011,2014-11-27 09:48:00,การเงิน-การลงทุน,คาดหุ้นเช้าแกว่งไซด์เวย์,โบรกฯ คาดหุ้นเช้านี้แกว่งไซด์เวย์ช่วงไร้ปัจจัย...,นายอภิชาติ ผู้บรรเจิดกุล ผู้อำนวยการสายงานวิเค...,https://www.bangkokbiznews.com/news/detail/620011


##2.2 Data Cleaning


In [ ]:
# Convert column heads to lower case
bkkbiz.columns.str.lower()

Index(['id', 'date', 'category', 'title', 'description', 'article', 'url'], dtype='object')

In [ ]:
# Check dtypes
bkkbiz.dtypes

ID                      int64
date           datetime64[ns]
category               object
title                  object
description            object
article                object
url                    object
dtype: object

In [ ]:
# Check shape
bkkbiz.shape

(361228, 7)

In [ ]:
# Check nulls
bkkbiz.isnull().sum()

ID             0
date           0
category       0
title          0
description    0
article        0
url            0
dtype: int64

In [ ]:
# Check duplicate rows
set([bkkbiz['title'].value_counts().count(),
     bkkbiz['description'].value_counts().count(), 
     bkkbiz['article'].value_counts().count(), 
     bkkbiz.shape[0]])

{337402, 352490, 354923, 361228}

In [ ]:
# Check deplicates
bkkbiz['article'].value_counts().count(), bkkbiz['article'].isnull().sum()

(337402, 0)

##2.3 Preprocessing

ลองเฉพาะหมวด digital ก่อน

In [ ]:
bkkbiz['category'].unique()

array(['ต่างประเทศ', 'การเงิน-การลงทุน', 'การเมือง', 'คุณภาพชีวิต-สังคม',
       'ธุรกิจ', 'ไลฟ์สไตล์', 'Bizweek', 'ทั่วไป', 'การเงิน', 'เศรษฐกิจ',
       'ไอที-นวัตกรรม', 'IT & gadget', 'ท่องเที่ยว', 'Art & living',
       'บันเทิง', 'ยานยนต์', 'อสังหาริมทรัพย์', 'ดูดวง', '@Taste',
       'ข่าวพระราชสำนัก', 'Special', 'จุดประกาย', 'Data Analysis',
       'ข่าวในพระราชสำนัก', 'นวัตกรรม', 'สุขภาพ-คุณภาพชีวิต',
       'สิ่งแวดล้อม', 'ข่าวทั่วไป', 'เศรษฐกิจ-ธุรกิจ', 'ประชาสัมพันธ์',
       'CEO Blog', 'คอลัมนิสต์', 'การศึกษา', 'ความงาม-แฟชั่น',
       'Training Course', 'Biz2U', 'หุ้น', 'Corporate Moves', 'ลอตเตอรี่',
       'สาธารณสุข', 'แรงงาน', 'เทคโนโลยี', 'คุณภาพชีวิต'], dtype=object)

In [ ]:
# Create economics news DataFrame
econ = bkkbiz[bkkbiz['category']=='เศรษฐกิจ']

In [ ]:
# Recheck
econ.shape

(35453, 7)

In [ ]:
# Recheck 
econ.head()

,ID,date,category,title,description,article,url
8993,632149,2015-01-30 18:20:00,เศรษฐกิจ,ซีพีเอฟร่วมลดก๊าซเรือนกระจก,ซีพีเอฟ ลงนามความร่วมมือโครงการต้นแบบลดก๊าซเรื...,นายสุนันต์ อรุณนพรัตน์(ที่ 4 จากซ้าย) ผู้ช่วยร...,https://www.bangkokbiznews.com/news/detail/632149
9409,632695,2015-02-03 16:30:00,เศรษฐกิจ,จับตายักษ์ใหญ่คว้า'โซลาร์'กองทัพบก,"จับตายักษ์ใหญ่คว้า""โซลาร์""กองทัพบก ปูดบ.รีนิวเ...",รายงานข่าวจากวงการพลังงาน เปิดเผยว่า จากการที่...,https://www.bangkokbiznews.com/news/detail/632695
10191,633724,2015-02-09 10:03:00,เศรษฐกิจ,หวั่น'ส่งออก'ทรุดต่อเนื่อง,นักเศรษฐศาสตร์-นักการเงิน ห่วงส่งออกไทยหดตัวต่...,"""กอบศักดิ์"" ชี้เงินบาทแข็งเกินพื้นฐาน ด้าน ""ตี...",https://www.bangkokbiznews.com/news/detail/633724
10194,633727,2015-02-09 10:20:00,เศรษฐกิจ,จี้'แบงก์รัฐ'ดันแพ็คเกจแก้จน,"กระทรวงคลัง เผย ""แบงก์รัฐ"" เร่งทำโครงการช่วยเห...",แหล่งข่าวจากกระทรวงการคลัง เปิดเผยว่า สถาบันกา...,https://www.bangkokbiznews.com/news/detail/633727
10205,633738,2015-02-09 11:02:00,เศรษฐกิจ,ดอลลาร์แข็งฉุด'หนี้นอก'ลด,"""แบงก์ชาติ"" เผยอดหนี้ต่างประเทศเดือนธ.ค. รวมกว...",รายงานข่าวจากธนาคารแห่งประเทศไทย (ธปท.) เปิดเผ...,https://www.bangkokbiznews.com/news/detail/633738


In [ ]:
# Create 
def token_1(text):
  stopwords = thai_stopwords()
  tokens = word_tokenize(text, keep_whitespace = False)
  tokens = [token for token in tokens if token not in stopwords]
  return tokens

In [ ]:
def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [token.strip('\'').strip('\"') for token in tokens] # remove quotations before/after token
    tokens = [token for token in tokens if token not in STOPWORDS] # remove stopwords
    tokens = [token for token in tokens if re.match(r'[A-Za-zก-๙0-9][A-Za-zก-๙0-9\.\-]+', token)] # token pattern : only words/digits
    return tokens


In [ ]:
# Tokenize & remove stopwords


In [ ]:
txt = 'อนิกจะได้ไปอยู่ bu ไหนนนน'

In [ ]:
print(re.search(r'\w+\d+', txt))

None


In [ ]:
print(re.match(r'[A-Za-zก-ฮ]', txt))

<re.Match object; span=(0, 1), match='อ'>


In [ ]:
print([cha for cha in txt if re.match(r'[A-Za-zก-ฮ]', txt)])

['อ', 'น', 'ิ', 'ก', 'จ', 'ะ', 'ไ', 'ด', '้', 'ไ', 'ป', 'อ', 'ย', 'ู', '่', ' ', 'b', 'u', ' ', 'ไ', 'ห', 'น', 'น', 'น', 'น']


In [ ]:
word_tokenize(txt)

['อ', 'นิ', 'ก', 'จะ', 'ได้', 'ไป', 'อยู่', ' ', 'bu', ' ', 'ไหน', 'นนน']

In [ ]:
print([word for word in txt if re.match(r'[A-Za-zก-ฮ]', txt)])

['อ', 'น', 'ิ', 'ก', 'จ', 'ะ', 'ไ', 'ด', '้', 'ไ', 'ป', 'อ', 'ย', 'ู', '่', ' ', 'b', 'u', ' ', 'ไ', 'ห', 'น', 'น', 'น', 'น']


In [ ]:
word_tokenize(test, ke)

['อ', 'นิ', 'ก', 'จะ', 'ได้', 'ไป', 'อยู่', ' ', 'bu', ' ', 'ไหน', 'นนน']

In [ ]:
tokens = word_tokenize(test, keep_whitespace = False)

In [ ]:
stopwords = thai_stopwords()

In [ ]:
tokens = [token for token in tokens if token not in stopwords]

In [ ]:
bkkbiz['article'][:10]

0    สัญญาน้ำมันดิบตลาดล่วงหน้าสหรัฐ ปิดตลาดปรับตัว...
1    สัญญาทองคำตลาดล่วงหน้าสหรัฐ ปิดปรับตัวลงเล็กน้...
2    สมาคมค้าทองคำรายงานราคาทอง 96.5% ประจำวันที่ 2...
3    นักบริหารเงินจากธนาคารพาณิชย์ เปิดเผยว่า เงินบ...
4    นายอภิชาติ ผู้บรรเจิดกุล ผู้อำนวยการสายงานวิเค...
5    รัฐมนตรีว่าการกระทรวงน้ำมันของอิหร่าน นายบิจาน...
6    เมื่อเวลา 23.00 น. วันที่ 26 พ.ย. ที่ สน.ลุมพิ...
7    นายเกรียงศักดิ์ บุญตาปวน หัวหน้าฝ่ายความมั่นคง...
8    นางสาวดวงกมล โชตะนา กรรมการผู้อำนวยการ บริษัท ...
9    พาณิชย์เผยส่งออกเดือนต.ค. ขยายตัว 3.97% ต่อเนื...
Name: article, dtype: object

In [ ]:
test = pd.DataFrame(bkkbiz['article'][:10000])
type(test)

pandas.core.frame.DataFrame

In [ ]:
test.head()

,article
0,สัญญาน้ำมันดิบตลาดล่วงหน้าสหรัฐ ปิดตลาดปรับตัว...
1,สัญญาทองคำตลาดล่วงหน้าสหรัฐ ปิดปรับตัวลงเล็กน้...
2,สมาคมค้าทองคำรายงานราคาทอง 96.5% ประจำวันที่ 2...
3,นักบริหารเงินจากธนาคารพาณิชย์ เปิดเผยว่า เงินบ...
4,นายอภิชาติ ผู้บรรเจิดกุล ผู้อำนวยการสายงานวิเค...


In [ ]:
test['article_tokenized'] = test['article'].apply(token_1)

In [ ]:
test.head()

,article,article_tokenized
0,สัญญาน้ำมันดิบตลาดล่วงหน้าสหรัฐ ปิดตลาดปรับตัว...,"[สัญญา, น้ำมันดิบ, ตลาด, ล่วงหน้า, สหรัฐ, ตลาด..."
1,สัญญาทองคำตลาดล่วงหน้าสหรัฐ ปิดปรับตัวลงเล็กน้...,"[สัญญา, ทองคำ, ตลาด, ล่วงหน้า, สหรัฐ, ปรับตัว,..."
2,สมาคมค้าทองคำรายงานราคาทอง 96.5% ประจำวันที่ 2...,"[สมาคม, ค้า, ทองคำ, รายงาน, ราคา, ทอง, 96.5, %..."
3,นักบริหารเงินจากธนาคารพาณิชย์ เปิดเผยว่า เงินบ...,"[บริหาร, เงิน, ธนาคารพาณิชย์, เงินบาท, ตลาด, เ..."
4,นายอภิชาติ ผู้บรรเจิดกุล ผู้อำนวยการสายงานวิเค...,"[อภิ, ชาติ, บรรเจิด, กุล, ผู้อำนวยการ, สายงาน,..."


In [ ]:
# Lemmatize/Stemming

#Part 3: Exploratory Data Analysis (EDA)
---

##3.1 Total Corpus

In [ ]:
# Write word_count() function
cols = ['title', 'description', 'article']

for col in cols:
  bkkbiz[col+'_word_count'] = bkkbiz[col].str.split().str.len()

In [ ]:
bkkbiz.head()

,ID,date,category,title,description,article,url,title_word_count,description_word_count,article_word_count
0,620001,2014-11-27 08:01:00,ต่างประเทศ,น้ำมันดิบปิดร่วง-คาดโอเปคไม่ลดกำลังผลิต,น้ำมันดิบปิดร่วงแตะระดับต่ำสุดในรอบ 4 ปี-คาดโอ...,สัญญาน้ำมันดิบตลาดล่วงหน้าสหรัฐ ปิดตลาดปรับตัว...,https://www.bangkokbiznews.com/news/detail/620001,1,3,47
1,620002,2014-11-27 08:20:00,ต่างประเทศ,ทองคำปิดร่วงหลังสหรัฐเผยข้อมูลเศรษฐกิจ,สัญญาทองคำตลาดล่วงหน้าสหรัฐ ปิดปรับตัวลงเล็กน้...,สัญญาทองคำตลาดล่วงหน้าสหรัฐ ปิดปรับตัวลงเล็กน้...,https://www.bangkokbiznews.com/news/detail/620002,1,3,31
2,620009,2014-11-27 09:34:00,การเงิน-การลงทุน,"ทองแท่งขายออก18,600บ.ไม่เปลี่ยนแปลง","ราคาทองคำวันนี้ทองแท่งขายออกบาทละ 18,600 บาท ท...",สมาคมค้าทองคำรายงานราคาทอง 96.5% ประจำวันที่ 2...,https://www.bangkokbiznews.com/news/detail/620009,1,7,29
3,620010,2014-11-27 09:36:00,การเงิน-การลงทุน,เงินบาทเปิด32.75/77คาดแกว่งแคบ,ค่าเงินบาทเปิดที่ 32.75/77 บาท/ดอลลาร์ คาดแกว่...,นักบริหารเงินจากธนาคารพาณิชย์ เปิดเผยว่า เงินบ...,https://www.bangkokbiznews.com/news/detail/620010,1,8,46
4,620011,2014-11-27 09:48:00,การเงิน-การลงทุน,คาดหุ้นเช้าแกว่งไซด์เวย์,โบรกฯ คาดหุ้นเช้านี้แกว่งไซด์เวย์ช่วงไร้ปัจจัย...,นายอภิชาติ ผู้บรรเจิดกุล ผู้อำนวยการสายงานวิเค...,https://www.bangkokbiznews.com/news/detail/620011,1,2,50


In [ ]:
# Plot word count with histogram

# Word_count column list


# Set subplots
fig, ax = plt.subplots(3, 1, figsize=(10,15))

# Create histogram of each word count columns by looping
for n, col in enumerate(cols_word_count):
    sns.histplot(data=df, 
                 x=col, 
                 bins=30, 
                 color='salmon', 
                 ax=ax[n]).\
                set_title(col, size=20)

plt.tight_layout()

##3.2 Test Corpus

In [ ]:
# title word count in test df
test['title_word_count'] = test['article'].str.split().str.len()
test

,article,article_tokenized,title_word_count
0,สัญญาน้ำมันดิบตลาดล่วงหน้าสหรัฐ ปิดตลาดปรับตัว...,"[สัญญา, น้ำมันดิบ, ตลาด, ล่วงหน้า, สหรัฐ, ตลาด...",47
1,สัญญาทองคำตลาดล่วงหน้าสหรัฐ ปิดปรับตัวลงเล็กน้...,"[สัญญา, ทองคำ, ตลาด, ล่วงหน้า, สหรัฐ, ปรับตัว,...",31
2,สมาคมค้าทองคำรายงานราคาทอง 96.5% ประจำวันที่ 2...,"[สมาคม, ค้า, ทองคำ, รายงาน, ราคา, ทอง, 96.5, %...",29
3,นักบริหารเงินจากธนาคารพาณิชย์ เปิดเผยว่า เงินบ...,"[บริหาร, เงิน, ธนาคารพาณิชย์, เงินบาท, ตลาด, เ...",46
4,นายอภิชาติ ผู้บรรเจิดกุล ผู้อำนวยการสายงานวิเค...,"[อภิ, ชาติ, บรรเจิด, กุล, ผู้อำนวยการ, สายงาน,...",50
...,...,...,...
9995,ผู้สื่อข่าวรายงานจากรัฐสภา ว่า นายเทียนฉาย กีร...,"[ผู้สื่อข่าว, รายงาน, รัฐสภา, เทียน, ฉาย, กี, ...",45
9996,ผู้สื่อข่าวรายงานว่าในการประชุมคณะกรรมาธิการ (...,"[ผู้สื่อข่าว, รายงาน, การประชุม, คณะกรรมาธิการ...",72
9997,พ.ต.อ.ศักดา เจริญกุล ผกก.สส.ภจว.สงขลาพร้อมเจ้า...,"[พ.ต.อ., ศักดา, เจริญ, กุล, ผกก., สส., ภ, จว.,...",75
9998,พล.อ.จรัล กุลละวณิชย์ อดีตเลขาธิการสภาความมั่น...,"[พล, ., อ., จรัล, กุล, วณิชย์, เลขาธิการ, สภา,...",63


In [ ]:
# article_tokenized_joined


In [ ]:
# Sum total word count
test['title_word_count'].sum()

1054972

In [ ]:
# Instantiate CountVectorizer
cvec = CountVectorizer(stop_words=thai_stopwords,
                      ngram_range=(1,1))

# Instantiate TfidfVectorizer
tfidf = TfidfVectorizer(stop_words=thai_stopwords,
                      ngram_range=(1,1))

In [ ]:
cvec.fit_transform(test['article_tokenized'])

TypeError: ignored

In [ ]:
# Fit&Transform with CountVectorizer -> DataFrame
cvec_1 = cvec.fit_transform(test['article_tokenized'])
cvec_1_df = pd.DataFrame(data_cvec_1.todense(), columns=cvec.get_feature_names_out())

# Fit&Transform with TfidfVectorizer -> DataFrame
tfidf_1 = tfidf.fit_transform(test['article_tokenized'])
tfidf_1_df = pd.DataFrame(data_tfidf_1.todense(), columns=tfidf.get_feature_names_out())

TypeError: ignored

#Part 4: Modeling
---

In [ ]:
# Create Corpus
dictionary = Dictionary(test['article_tokenized'])
corpus = [dictionary.doc2bow(text) for text in test['article_tokenized']]

In [ ]:
# Model
def lda_model(num_topics=None, topicid=None):
  lda = LdaModel(corpus, num_topics=num_topics)
  topic = pd.DataFrame(lda_1.get_topic_terms(topicid=1, topn=20)).rename(columns={0:'index', 1:'probability'})
  top_20 = [dictionary[index] for index, prob in lda.get_topic_terms(topicid=topicid, topn=20)]
  return top_20


In [ ]:
lda_model(num_topics=5, topicid=0)

["'",
 '-',
 '"',
 'ไทย',
 'ทีม',
 ',',
 '2',
 '1',
 '3',
 'ปี',
 'วันที่',
 ')',
 '(',
 'โควิด',
 'คน',
 'ล้าน',
 'บาท',
 '4',
 '19',
 'ทำ']

In [ ]:
# Instantiate model
lda_1 = LdaModel(corpus, num_topics = 3)   

In [ ]:
# See words in the 1st topic
topic_1 = pd.DataFrame(lda_1.get_topic_terms(topicid=1, topn=20)).rename(columns={0:'index', 1:'probability'})
topic_1

,index,probability
0,1,0.031594
1,0,0.013589
2,2,0.011026
3,279,0.005958
4,101,0.005512
5,37,0.005236
6,470,0.005064
7,78,0.004900
8,51,0.004546
9,34856,0.004383


In [ ]:
topic_1 = topic_1.rename(columns={0:'index', 1:'probability'})
type(topic_1)

pandas.core.frame.DataFrame

In [ ]:
topic_1_test = lda_1.show_topics(formatted=False)
topic_1_test

[(0,
  [('1', 0.021037055),
   ('0', 0.012165701),
   ('621', 0.007095062),
   ('98', 0.006489503),
   ('99', 0.0061611314),
   ('69', 0.0058810534),
   ('2', 0.005835319),
   ('231', 0.005793298),
   ('101', 0.005258681),
   ('279', 0.0051404457)]),
 (1,
  [('1', 0.031594224),
   ('0', 0.01358872),
   ('2', 0.011025867),
   ('279', 0.005958481),
   ('101', 0.0055115186),
   ('37', 0.0052360413),
   ('470', 0.0050639906),
   ('78', 0.004900142),
   ('51', 0.004546382),
   ('34856', 0.0043827933)]),
 (2,
  [('1', 0.034448985),
   ('113', 0.013822185),
   ('37', 0.013489862),
   ('97', 0.013275735),
   ('209', 0.011586499),
   ('2', 0.010790966),
   ('46', 0.010138918),
   ('98', 0.009765995),
   ('99', 0.009626163),
   ('0', 0.007961102)])]

In [ ]:
top20_1 = [dictionary[index] for index, prob in lda_1.get_topic_terms(topicid=0, topn=20)]

In [ ]:
top20_1

["'",
 '"',
 '.',
 '(',
 ')',
 'เรื่อง',
 '-',
 'ไทย',
 '1',
 'คน',
 '2',
 '3',
 'ทำ',
 'ประเทศ',
 '/',
 'ปี',
 'พื้นที่',
 'ประชาชน',
 'อ.',
 '5']

In [ ]:
top20_2 = [dictionary[index] for index, prob in lda_1.get_topic_terms(topicid=1, topn=20)]

top20_2

["'",
 '"',
 '-',
 'คน',
 '1',
 'ปี',
 '2',
 '3',
 'วันที่',
 'โควิด',
 'ทีม',
 ')',
 'ไทย',
 ',',
 'ล้าน',
 'บาท',
 '4',
 'เรื่อง',
 'เดินทาง',
 '(']

In [ ]:
top20_3 = [dictionary[index] for index, prob in lda_1.get_topic_terms(topicid=2, topn=20)]

top20_3

["'",
 'บาท',
 'ปี',
 '%',
 'ล้าน',
 '-',
 'ราคา',
 '(',
 ')',
 '"',
 'ไทย',
 'คาด',
 'บริษัท',
 '3',
 'ลดลง',
 'หุ้น',
 'ซื้อ',
 'ตลาด',
 '1',
 'เศรษฐกิจ']

In [ ]:
# Evaluation: perplexity

#Part 5: Final Model 
---

#Part 6: Analysis
---

# Part 7: Interpretation
---

# Part 8: Conclusion
---